## Project 6

In [1]:
import os
import subprocess
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
import scipy
import requests
from imdbpie import Imdb
import nltk
import matplotlib.pyplot as plt
import urllib
from bs4 import BeautifulSoup
import nltk
import collections
import re
import csv
import psycopg2

%matplotlib inline

### Pre-Work: Write a problem statement 

## Part 1: Acquire the Data

#### 1. Connect to the IMDB API

In [2]:
imdb = Imdb()
imdb = Imdb(anonymize=True)

#### 2. Query the top 250 rated movies in the database

In [3]:
movies = pd.DataFrame(imdb.top_250())

In [4]:
movies.head()

,can_rate,image,num_votes,rating,tconst,title,type,year
0,True,{u'url': u'https://images-na.ssl-images-amazon...,1730915,9.3,tt0111161,The Shawshank Redemption,feature,1994
1,True,{u'url': u'https://images-na.ssl-images-amazon...,1182761,9.2,tt0068646,The Godfather,feature,1972
2,True,{u'url': u'https://images-na.ssl-images-amazon...,811039,9.0,tt0071562,The Godfather: Part II,feature,1974
3,True,{u'url': u'https://images-na.ssl-images-amazon...,1715862,9.0,tt0468569,The Dark Knight,feature,2008
4,True,{u'url': u'https://images-na.ssl-images-amazon...,462049,8.9,tt0050083,12 Angry Men,feature,1957


#### 3. Only select the top 25 movies and delete the uncessary rows

In [7]:
df = movies[0:26]

In [8]:
del df['can_rate']
del df['image']
del df['type']

In [8]:
df.head()

,num_votes,rating,tconst,title,year
0,1633889,9.3,tt0111161,The Shawshank Redemption,1994
1,1118799,9.2,tt0068646,The Godfather,1972
2,762879,9.0,tt0071562,The Godfather: Part II,1974
3,1616346,9.0,tt0468569,The Dark Knight,2008
4,835155,8.9,tt0108052,Schindler's List,1993


#### 4. Write the Results to a csv

In [47]:
df.to_csv('movies.csv')

## Part 2: Wrangle the text data

#### 1. Convert the listing identification numbers (tconst) from the first dataframe to a list

In [12]:
numid = df['tconst'].tolist()

In [13]:
print(numid)

['tt0111161', 'tt0068646', 'tt0071562', 'tt0468569', 'tt0108052', 'tt0050083', 'tt0110912', 'tt0167260', 'tt0060196', 'tt0137523', 'tt0120737', 'tt0080684', 'tt0109830', 'tt1375666', 'tt0167261', 'tt0073486', 'tt0099685', 'tt0133093', 'tt0047478', 'tt0076759', 'tt0317248', 'tt0114369', 'tt0102926', 'tt0038650', 'tt0114814', 'tt0118799']


#### 2. Scrape the reviews for the top 25 movies

In [29]:
for x in (numid):
    address = ('http://www.imdb.com/title/' + x + '/reviews')
    html = requests.get(address).text
 'html.parser')

#### 3. Work through each title and find the most common descriptors

*Hint*: "soup" from BeautifulSoup is the html returned from all 25 pages. You'll need to either address each page individually or break them down by elements

In [15]:
title = soup.find("title")
page = soup.find_all('p')

In [18]:
print(title)

<title>Life Is Beautiful  Reviews &amp; Ratings - IMDb</title>


In [17]:
print(page)

[<p class="navCategory">
<a href="/movies-in-theaters/?ref_=nv_tp_inth_1">Movies</a>,
                <a href="/chart/toptv/?ref_=nv_tp_tv250_2">TV</a><br/>
                &amp; <a href="/showtimes/?ref_=nv_tp_sh_3">Showtimes</a></p>, <p class="navCategory">
<a href="/search/name?gender=male,female&amp;ref_=nv_tp_cel_1">Celebs</a>,
                <a href="/awards-central/?ref_=nv_tp_awrd_2">Events</a><br/>
                &amp; <a href="/gallery/rg784964352?ref_=nv_tp_ph_3">Photos</a></p>, <p class="navCategory">
<a href="/news/top?ref_=nv_tp_nw_1">News</a> &amp;<br/>
<a href="/boards/?ref_=nv_tp_bd_2">Community</a></p>, <p class="navCategory singleLine watchlist"><a href="/list/watchlist?ref_=nv_wl_all_0">Watchlist</a>
</p>, <p class="navCategory">
<a href="http://pro.imdb.com/signup/index.html?rf=cons_nb_hm&amp;ref_=cons_nb_hm">
<img alt="IMDbPro Menu" src="http://i.media-imdb.com/images/SF4a741137cf9a260e127fef64455ebfbc/navbar/imdbpro_logo_nb.png"/>
</a>
</p>, <p>Industry informa

#### 4. Convert to a string and remove the non AlphaNumeric characters

In [22]:
foo = str(page)

In [23]:
regex = re.compile('[^a-zA-Z]')
new = regex.sub(' ', foo)

#### 5. Tokenize the Output

In [24]:
tokens = nltk.word_tokenize(new)